## Generative AI - Inference(via LangChain)

In [ ]:
from os import getenv
from dotenv import load_dotenv

from langchain_community.llms.oci_generative_ai import OCIGenAI

load_dotenv()

COMPARTMENT_ID = getenv("COMPARTMENT_ID")
SERVICE_ENDPOINT = getenv("GEN_AI_INFERENCE_ENDPOINT") # https://inference.generativeai.us-chicago-1.oci.oraclecloud.com

### Using API Key

In [ ]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command",
    auth_profile="CHICAGO",
    model_kwargs={
        'max_tokens': 100
    }
)

response = llm.invoke("Tell me a joke", temperature=0.7)

print(response)

In [ ]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command-light",
    auth_profile="CHICAGO"
)

response = llm.invoke("Tell me a joke", temperature=0.7)

print(response)

In [ ]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="meta.llama-2-70b-chat",
    auth_profile="CHICAGO"
)

response = llm.invoke("Tell me a joke", temperature=0.7)

print(response)

### Using Instance Principal

In [ ]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command",
    # ... omit ...
    auth_type="INSTANCE_PRINCIPAL"
)

response = llm.invoke("Tell me a joke", temperature=0.7)
print(response)

### Using Session Token

In [ ]:
llm = OCIGenAI(
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command",
    auth_type="SECURITY_TOKEN",
    auth_profile="langchain"
)

response = llm.invoke("Tell me a joke", temperature=0.7)
print(response)

## Generative AI - Embeddings

In [ ]:
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings

In [ ]:
embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-english-v3.0",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID
)

response = embeddings.embed_query(text="This is a query in English.")
print(response)

In [ ]:
response = embeddings.embed_documents(
    texts=[
        "This is a first document in English",
        "This is a second document in English"
    ]
)

print(response)

## Prompt Template

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

llm_cohere = OCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    model_id="cohere.command"
)

prompt = PromptTemplate(input_variables=["query"], template="{query}")
llm_chain = LLMChain(llm=llm_cohere, prompt=prompt)

response = llm_chain.invoke("what is the capital of france?")

response

## Simple RAG

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

In [ ]:
embeddings = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    model_id="cohere.embed-english-v3.0",
    compartment_id=COMPARTMENT_ID
)

vectorstore = FAISS.from_texts(
    [
        "Larry Ellison co-founded Oracle Corporation in 1977 with Bob Miner and Ed Oates.",
        "Oracle Corporation is an American multinational computer technology company headquartered in Austin, Texas, United States.",
    ],
    embedding=embeddings
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template=template)
llm_cohere = OCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=SERVICE_ENDPOINT,
    model_id="cohere.command",
    compartment_id=COMPARTMENT_ID
)

chain = (
    { "context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm_cohere
    | StrOutputParser()
)

print(chain.invoke("when was oracle founded?"))
print(chain.invoke("where is oracle headquartered?"))